# 财经新闻分析与分类

In [18]:
# 常量定义
# 新闻爬取xlxs文件目录：
NEWS_PATH = '../news/'

#语料库文件路径：
DATA_PATH = '../data/news_words.txt'

## 财经新闻抓取数据导入

对于爬虫抓取的数据，首先将Excel格式文件转换为 utf-8 编码的CSV文件格式。


格式：CSV文件
* 新闻标题
* 新闻时间
* 新闻正文



In [1]:
import pandas as pd
import os  
import os.path  
  
#这里放着你要操作的文件夹名称  
#datapath = '../news/'  

# 将给定路径path的xlsx 文件转换为CSV 文件，并存在相同目录下
def xlsx_to_csv_pd(path):
    if 'xlsx' in path:
        csvPath = path.replace('xlsx','csv')
    elif 'xls' in path:
        csvPath = path.replace('xls','csv')
    else:
        return None
    try:
        data_xls = pd.read_excel(path, index_col=0)
        data_xls.to_csv(csvPath, encoding='utf-8')
    except:
        print('Error changing the following file from xls to csv:',path)


In [4]:
xlsx_to_csv_pd("../data/news0312.xlsx")

In [3]:

#把目录下的文件名全部获取保存在files中  
files = os.listdir("../data/")  
for fname in files:
    fpath = "../data/" + fname
    xlsx_to_csv_pd(fpath)
xlsx_to_csv_pd()

TypeError: xlsx_to_csv_pd() missing 1 required positional argument: 'path'

## csv 文档分词后转换
读取所有CSV文件中的新闻，对于每条内容进行分词后，写成新文件的一行，加以存储。

In [13]:
import jieba
# 加载停用词，输入停用词文件，输出停用词list
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords

temp_stop_list = ['\u3000','\xa0','\t']
stop_words = stopwordslist("../utils/stopwords.txt") + temp_stop_list
# 为结巴分词词库加载股票名词汇
jieba.load_userdict('../data/user_dict.txt')

# check if a string s is a number
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False
 
## jieba分词：输入string & 停用词文件，输出分词结果list
def jieba_split(content):
    '''
    content: 输入文本（string）
    stop_path: 停用词字典文件路径（string）
    返回：list，jieba分词结果
    '''
    str_content = str(content).replace('\t', '').replace('\n', '').replace(' ','')
    str_words = ','.join(jieba.cut_for_search(str_content)).split(",")
    ret_list = []
    for word in str_words:
        if word not in stop_words:
            if word[-1] != '%':
                ret_list.append(word)
    return ret_list


# 测试：读取一个csv文件，将列名替换：“标题”：title，‘正文’：content

# write_news_words
# 输入 csv 文件的 file_path
# 操作：分词-写入DATA_FILE
def write_news_words(file_path):
    file_data = pd.read_csv(file_path)
    file_data.rename(columns={'标题':'title', '正文':'content','正文1':'content',"字段1_文本":"title"}, inplace = True)
    file = open(DATA_PATH,"a")
    for index, row in file_data.iterrows():
        #print(row.content)
        row_words = jieba_split(str(row.title) + str(row.content))
        file_words = " ".join(row_words)
        file.write(file_words + ' \n')
    file.close()
    

    
# 添加文件到语料库
# 输入：new_path 为新增的抓取csv数据的文件夹
# 执行：将csv文件的分词结果写入语料库文件
def append_csv2txt(new_path):
    files = os.listdir(new_path)
    print(files)
    for fname in files:
        fpath = new_path + fname
        if 'csv' in fpath:
            print(fpath)
            write_news_words(fpath)
    



            
            

In [15]:
#append_csv2txt('../news/')
#append_csv2txt('../labels/')
append_csv2txt('../tmp/')
print("Finish!")

['东财关键词对应新闻-0427.csv', '东方财经关键词板块对应新闻-0427.csv']
../tmp/东财关键词对应新闻-0427.csv
../tmp/东方财经关键词板块对应新闻-0427.csv
Finish!


In [9]:
file_data = pd.read_csv("../labels/第一财经板块对应新闻-0416.csv")

## 载入语料库数据

读取对应的语料库文件，将每一行读入后形成list，载入doc2vec


## Doc2Vec

采用 Doc2Vec 方法将一段文字转换成对应的向量，各文本之间的差别可以用向量夹脚进行计算。
Doc2Vec 的目的是获得文档的一个固定长度的向量表达。

数据：多个文档，以及它们的标签，可以用标题作为标签。 
影响模型准确率的因素：语料的大小，文档的数量，越多越高；文档的相似性，越相似越好。

优势：Doc2Vec是无监督学习，不需要具体标注，可以用文档名作为标签。
注意事项：训练集越大，结果越好。语料库中各文章越相似，结果越好


使用方式：gensim 的 doc2vec 

后续工作：对于各文章的向量，首先可以进行基于语义的归类、相似度计算。其次可以将向量与关键词进行匹配，方法可以是标记训练，也可以是关键词 -- 搜索文本 -- doc2Vec ，之后计算向量夹角。


In [8]:
import jieba  
import sys  
from collections import namedtuple
from gensim.models import doc2vec


docs = []

    
file = open(DATA_PATH,'r')
for (tag,line) in enumerate(file):
    words = line.split(" ")
    words.remove("\n")
    tags = [tag]
    docs.append(analyzedDocument(words, tags))

model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

# Get the vectors
print(model.dovcecs[0])


NameError: name 'analyzedDocument' is not defined

In [1]:

# model.docvecs[0]
# model.docvecs[1000]
#model.infer_vector(['大数据'])
len(model.docvecs)


def test():  
    model_dm = model 
    test_text = docs[13586].words
    #test_text = ["区块链"]
    print(test_text)
    inferred_vector_dm = model_dm.infer_vector(test_text)  
    print(inferred_vector_dm)
    sims = model_dm.docvecs.most_similar([inferred_vector_dm], topn=10)  
    return sims
    

sims = test()  

for count, sim in sims:  
    sentence = docs[count]  
    words = ''  
    for word in sentence[0]:  
        words = words + word + ''  
    print(sim)
    print(words, sim, len(sentence[0]))
    print("----------")

NameError: name 'model' is not defined

## 板块概念关键词获取
通过同花顺网站 http://q.10jqka.com.cn/gn/ ，获取概念关键词列表。
后续列表将根据华鑫的语料库和客户的需求进行扩充和更新

In [3]:
# Get notion list
# From: http://q.10jqka.com.cn/gn/
import re

file_object = open('../data/tonghuashun_gn.txt')
try:
     gn_text = file_object.read( )
finally:
     file_object.close()

gn_list = re.findall(r">(.+?)</a>", gn_text)
print(gn_list)

['阿里巴巴概念', '安防', '白酒', '白马股', '参股保险', '参股民营银行', '参股券商', '参股360', '参股新三板', '草甘膦', '超导', '超级品牌', '车联网', '充电桩', '创投', '大飞机', '大数据', '锂电池', '电子发票', '电子竞技', '电子商务', '电子信息', '迪士尼', '地下管网', '电力改革', '东盟自贸区', '二胎概念', '二维码识别', '风电', '分散染料', '氟化工', '福建自贸区', '高端装备', '高送转', '高铁', '高校', '工业4.0', '供应链金融', '股权转让', '广东自贸区', '光伏概念', '固废处理', '国产软件', '军工', '共享单车', '海工装备', '航运', '杭州亚运会', '核电', '互联网+', '互联网彩票', '互联网金融', '互联网医疗', '黄金', '沪港通概念', '互联网保险', '金融IC', '集成电路', '建筑节能', '家用电器', '节能环保', '节能照明', '金改', '京津冀一体化', '机器人概念', '基因测序', '健康中国', '军民融合', '举牌', '可燃冰', '跨境电商', '宽带中国', '蓝宝石', '冷链物流', '两桶油改革', '量子通信', '生态农业', '蚂蚁金服概念', '煤化工', '美丽中国', '民营医院', 'MSCI概念', '马云概念', '能源互联网', '农村电商', '农机', '农业现代化', 'O2O概念', 'OLED', 'P2P概念', '啤酒', '苹果概念', 'PM2.5', 'PPP概念', '汽车电子', '期货概念', '氢燃料电池', '禽流感', '区块链', '人工智能', '人脸识别', '融资融券', '乳业', '上海国资改革', '上海自贸区', '深港通', '生物医药', '石墨电极', '石墨烯', '食品安全', '首发新股', '手机游戏', '水利', '水泥', 'ST板块', '生物质能', '深圳国资改革', '钛白粉', '太阳能', '碳纤维', '特钢', '特高压', '腾讯概念', '特色小镇', '特斯拉', '天津自贸区', '天然气', '体育产业

## 新闻分词

我们使用jieba分词对每条新闻进行分词处理。

输入：文本（string)
输出：分词结果（list）

后续需要进行的工作包括：

* 增加财经类专有词汇进入jieba的词典
* 增加停用词词典，以过滤无效分词。


In [4]:
# Word Count

# wordcount方法，输入分词结果的list，对于每个单词进行数量统计

def wordcount(word_list):
    '''
    input: l as a list of strings
    output: dict of {word:count}
    '''
    count = {}
    for i in word_list:
        if i in count.keys():
            count[i] += 1
        else:
            count[i] = 1
    return count

# word_key_count 方法，输入分词结果list，对于特定的keywords中的词语进行统计（非keyword直接忽略）

def word_key_count(wordlist,keywords):
    '''
    input: wordlist as a list of strings
        keywords as a list of keywords
    output: dict of {keyword:count}
    '''
    count = {}
    for i in wordlist:
        if i in keywords:
            if i in count.keys():
                count[i] += 1
            else:
                count[i] = 1
    return count

In [149]:
## 对于目前的所有新闻，每条新闻进行关键词对应，查看关键词覆盖情况

file = open(DATA_PATH,'r')

#news_gn = pd.DataFrame(columns = ["news_id"] + gn_list)

for (tag,line) in enumerate(file):
    words = line.split(" ")
    for word in words:
        if word in gn_list:
            if word in gn_count.keys():
                gn_count[word] += 1
            else:
                gn_count[word] = 1
    #key_count = word_key_count(words,gn_list)
    #print(key_count)
    #print("----------")
print(gn_count)



{'债转股': 5682, '电子商务': 3002, '在线教育': 1132, '生物质能': 139, '稀缺资源': 286, '集成电路': 3597, '特斯拉': 7089, '蓝宝石': 326, '禽流感': 131, '家用电器': 4422, '网约车': 194, '超导': 398, '文化传媒': 1001, '核电': 6585, '手机游戏': 102, '锂电池': 3288, '草甘膦': 370, 'OLED': 1017, '太阳能': 2185, '特钢': 6309, '迪士尼': 2185, '黄金': 17328, '农机': 1602, '人工智能': 7496, '碳纤维': 261, '白酒': 4906, '可燃冰': 227, 'PM2.5': 387, '电子竞技': 140, '网络安全': 5330, '无人驾驶': 1685, '4G5G': 6, '建筑节能': 78, '创投': 4642, '虚拟现实': 1305, '污水处理': 869, '智能家居': 1073, '无人机': 1747, '人脸识别': 591, '电子信息': 2931, '水利': 5150, '啤酒': 39109, '国产软件': 1950, '军工': 41670, '生态农业': 126, '食品安全': 661, '举牌': 8110, '煤化工': 819, '安防': 1516, '金改': 86, '高铁': 3674, '航运': 3060, '特高压': 1395, '网络游戏': 435, '乳业': 3756, '石墨电极': 170, '水泥': 16772, '风电': 4390, '钛白粉': 1107, '分散染料': 107, '摘帽': 723, '天然气': 4591, '医疗器械': 1860, '猪肉': 697, '体育产业': 1911, '生物医药': 1216, '新能源': 24052, '白马股': 3082, '高校': 1130}


In [148]:

from pyecharts import Bar

attr = list(gn_count.keys())
v1 = list(gn_count.values())
bar = Bar("Bar chart", "precipitation and evaporation one year")
bar.add("precipitation", attr, v1, mark_line=["average"], mark_point=["max", "min"])
bar

In [139]:
d1 = {'集成电路': 1}
d2 = {'军工': 1, '人工智能': 1, '新能源': 2, '高校': 1, '集成电路': 1}

def 

TypeError: unsupported operand type(s) for +: 'dict_items' and 'dict_items'

In [5]:
## jieba
# imput：string
# output: list of words

import jieba  

news_list = test_data.loc[1:10,].content.tolist()
for news in news_list:
    news = str(news).replace('\t', '').replace('\n', '').replace(' ','')
    #seg = jieba.cut(news, cut_all=False)
    #print((" ".join(seg)))
    news_jieba = ','.join(jieba.cut_for_search(news)).split(",")
    #news_words = ','.join(jieba.cut_for_search(news)).split(",")
    news_keywords = word_key_count(news_jieba,gn_list)
    #print(news_keywords)
    #print("-----------------")
    

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.833 seconds.
Prefix dict has been built succesfully.


从上述结果可以看到，同花顺的板块关键词与新闻中能够分词得到的关键词之间有很大的距离，很多新闻不包含板块列表中的任何一个关键词。因而，不能单纯的以同花顺板块关键词作为文章的特征向量。

In [36]:
import jieba  
import sys  
from collections import namedtuple
from gensim.models import doc2vec


news_contents = test_data.loc[1:10,].content.tolist()
news_titles = test_data.loc[1:10,].title.tolist()
# 加载停用词
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
stop_words = stopwordslist("../utils/stopwords.txt")


def jieba_split(content):
    '''
    content: 输入string
    返回：list，jieba分词结果
    '''
    str_content = str(content).replace('\t', '').replace('\n', '').replace(' ','')
    words = ','.join(jieba.cut_for_search(news)).split(",")
    ret_list = []
    for word in words:
        if word not in stop_words:
            if word != '\t':
                ret_list.append(word)
    return ret_list


docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
# 对于读取的csv中的每行，title直接作为ID，content进行分词
for index, row in test_data.iterrows():
    words = jieba_split(row.content)
    tags = [index]

    docs.append(analyzedDocument(words, tags))

    
model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

# Get the vectors

model.docvecs[0]
model.docvecs[1]




/home/inesa/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/home/inesa/.local/lib/python3.5/site-packages/ipykernel_launcher.py:49: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


KeyError: "word '农业银行公告，拟以非公开发行方式发行不超过274.73亿股A股' not in vocabulary"

In [39]:
model.infer_vector(['大数据'])


array([-5.4359948e-04,  2.4775974e-03, -3.3414126e-03,  2.0195886e-03,
        3.1975079e-03, -2.3209287e-03, -4.8111733e-03, -1.0556693e-04,
       -4.6941759e-03,  8.6439901e-04, -6.0576486e-04,  3.6755938e-03,
        4.7860360e-03, -3.5068085e-03, -1.9612887e-03,  6.6922337e-04,
        3.9420077e-03,  2.2369092e-03, -3.0673554e-03,  3.7408150e-03,
       -1.4501261e-03,  4.6342192e-03,  2.3098721e-03, -4.4978056e-03,
        4.6808324e-03,  4.4101542e-03, -1.8619066e-04,  3.7129037e-03,
        1.0040315e-03,  4.2312015e-03,  1.7201813e-03,  2.8457702e-04,
        2.2850025e-03, -7.9294865e-04, -1.8799296e-03,  3.7282121e-03,
       -3.3937453e-03,  1.0326292e-03, -3.0222947e-03, -3.7595741e-03,
       -2.7006185e-03, -1.3564046e-03,  3.5538755e-03, -1.8512841e-03,
       -4.6294578e-03, -4.5664459e-03,  3.4132274e-03,  5.4600608e-04,
        4.6899985e-03,  2.8150952e-03,  1.4626229e-03,  1.9676671e-03,
        4.9300781e-03,  5.0489849e-05, -6.9381681e-04, -2.6175675e-03,
      

In [ ]:
# Import libraries

from gensim.models import doc2vec
from collections import namedtuple

# Load data

doc1 = ["This is a sentence", "This is another sentence"]

# Transform data (you can add more data preprocessing steps) 

docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(doc1):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))

# Train model (set min_count = 1, if you want the model to work with the provided example data set)

model = doc2vec.Doc2Vec(docs, size = 100, window = 300, min_count = 1, workers = 4)

# Get the vectors

model.docvecs[0]
model.docvecs[1]